<a href="https://colab.research.google.com/github/AasthathecoderX/Q-A-over-document-using-langchain/blob/main/Langchain_File_QA_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community #Different doc uploaded
!pip install faiss-cpu #facebook AI similarity search
!pip install pydf python-docx
!pip install sentence-transformers #as dataset is in particular sentence or txt
!pip install transformers #after sentence-transforms as first install for specific then for all types

In [ ]:
from google.colab import files #picks file from local and stores in Jupiter kernel
uploaded=files.upload()

file_path=list(uploaded.keys())[0]
print("Uploaded:",file_path)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader,TextLoader,Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


if file_path.endswith('.pdf'):
    loader=PyPDFLoader(file_path)
elif file_path.endswith('.docx') or file_path.endswith('.doc'):
    loader=Docx2txtLoader(file_path)
else:
    loader=TextLoader(file_path)

docs =loader.load()

#split into smaller chunks

splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
documents=splitter.split_documents(docs)

print(f"Total Chunks:{len(documents)}")

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embeddings)

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

flan_pipeline=pipeline("text2text-generation",
                       model="google/flan-t5-base",
                       max_length=520)#limit number of tokens of response
llm=HuggingFacePipeline(pipeline=flan_pipeline)

In [ ]:
from langchain.chains import RetrievalQA
qa=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",
                                retriever=vectorstore.as_retriever(search_kwargs={"k":3}))

query ="Give me short summarise the document"
print(qa.run(query))

In [ ]:
while True:
  q=input("Ask a question(or 'exit'):")
  if q.lower()=="exit":
    break
  print("Answer:",qa.run(q))